<a href="https://colab.research.google.com/github/boltonhowes22/image-segmentation-ehg/blob/master/week-00/Week_00_MNIST_Bolton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# First we are going to make sure we are using the most up to date version of tensorflow
try:
# %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
# Tensorboard is a cool visualization component of tensorflow. 
# helps keep track of training metrics
%load_ext tensorboard

# Import tensorflow and datetime and TensorBoard
import tensorflow as tf
import datetime
from keras.callbacks import TensorBoard
from keras.utils import to_categorical


# Clear any logs from previous runs
!rm -rf ./logs/ 

TensorFlow 2.x selected.


Using TensorFlow backend.


In [5]:
# Load Dataset
mnist = tf.keras.datasets.mnist

# Split Dataset into training and testing
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

## Need to reshape for convolutional neural network
# reshape data to fit model
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

# Define number of classes--MNIST has 10
num_classes = 10

# one-hot encode target
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Properly shapes the data if it is messed up
if tf.keras.backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Create a function to build a model 
def create_model():
  return tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, kernel_size=(11, 11), activation='relu', input_shape=input_shape, name='conv1'),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128,activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes,activation='sigmoid')],
            
            )

  # Hyperparameters
num_epochs = 15   # An epoch is the number of times the network will see each instance of training data
batch_size = 256
optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = 'accuracy'
#initial_learning_rate = 0.0008
#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate,
#    decay_steps=round(60000/batch_size),
#    decay_rate=0.9,
#    staircase=True)

#optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, amsgrad=False)


In [26]:
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./log', histogram_freq=1,
#                        write_graph=True,
#                        write_grads=True, 
#                        #batch_size=batch_size,
#                        write_images=True)
#
#model
round(60000/128)


469

In [27]:
model = create_model()
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metrics])


log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir ) # , histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=num_epochs,
          batch_size=batch_size, 
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 91s 2ms/sample - loss: 0.4501 - accuracy: 0.8713 - val_loss: 0.0742 - val_accuracy: 0.9760
Epoch 2/15
60000/60000 [==============================] - 90s 2ms/sample - loss: 0.1248 - accuracy: 0.9656 - val_loss: 0.0460 - val_accuracy: 0.9848
Epoch 3/15
60000/60000 [==============================] - 90s 1ms/sample - loss: 0.0870 - accuracy: 0.9753 - val_loss: 0.0380 - val_accuracy: 0.9875
Epoch 4/15
60000/60000 [==============================] - 90s 2ms/sample - loss: 0.0679 - accuracy: 0.9809 - val_loss: 0.0324 - val_accuracy: 0.9893
Epoch 5/15
60000/60000 [==============================] - 90s 1ms/sample - loss: 0.0576 - accuracy: 0.9833 - val_loss: 0.0296 - val_accuracy: 0.9905
Epoch 6/15
60000/60000 [==============================] - 90s 1ms/sample - loss: 0.0485 - accuracy: 0.9855 - val_loss: 0.0275 - val_accuracy: 0.9914
Epoch 7/15
60000/60000 [==============================] 

In [0]:
%tensorboard --logdir logs/fit